In [2]:
!pip install open-interpreter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 2.2 MB/s eta 0:00:001.8 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of litellm to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of litellm to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 18.6 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.9/249.9 kB 11.5 MB/s eta 0:00:0031m40.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from interpreter import OpenInterpreter

In [2]:
def print_agent_attrs(agent):
    # Get the dictionary of attributes
    attributes = agent.__dict__
    
    # Create a list of tuples containing key and type of value
    attributes_with_types = [(key, type(value).__name__) for key, value in attributes.items()]
    
    # Print the result
    for key, value_type in attributes_with_types:
        print(f"{key}: {value_type}")

In [12]:
def util_read_file(filename):
    with open(filename, 'r') as f:
        content = f.read()
    return content

def get_prompt(name):
    filepath = f"prompts/{name}.txt"
    return util_read_file(filepath)

In [4]:
import time

class FIFOQueue:
    def __init__(self):
        self.queue = []

    def enqueue(self, items):
        timestamp = time.time()
        if not isinstance(items, list):
            items = [items]
        self.queue.append({"timestamp": timestamp, "messages": items})

    def dequeue(self):
        if not self.is_empty():
            return self.queue.pop(0)
        else:
            raise IndexError("Dequeue from an empty queue")

    def is_empty(self):
        return len(self.queue) == 0

    def size(self):
        return len(self.queue)


In [18]:
class Agent:
    def __init__(self, name, special_commands=""):
        self.name = name
        self.states = ["SPECTATE", "MUTATE", "DICTATE"]
        self.state = "SPECTATE"
        self.inbox = FIFOQueue()
        self.internal_state = []
        self.oi = OpenInterpreter()
        # self.oi.loop = True
        self.oi.llm.model = "gpt-4o-mini"
        self.oi.llm.context_window = 128000
        self.oi.llm.max_tokens = 10000
        self.oi.system_message += special_commands
        self.oi.auto_run = True
        # self.vanilla_chat = None
        self.other_agents = []
        self.generated_new_messages = []
        self.most_recently_received_messages = []

    def add_other_agent(self, agent):
        self.other_agents.append(agent)

    def handle_state(self):
        print(f"[{self.name}] Current State: {self.state}")
        print(f"[{self.name}] Inbox Size: {self.inbox.size()}")

        if self.state == "SPECTATE":
            if not self.inbox.is_empty():
                self.most_recently_received_messages = self.inbox.dequeue()["messages"]
                print(f"[{self.name}] Received messages: {self.most_recently_received_messages}")
                self.state = "MUTATE"

        elif self.state == "MUTATE":
            self.internal_state.extend(self.most_recently_received_messages)
            print(f"[{self.name}] Internal state after mutation: {self.internal_state}")
            messages = [{"role": "user", "content": msg["content"], "type": "message"} for msg in self.most_recently_received_messages]
            self.generated_new_messages = self.oi.chat(messages, display=True, stream=False, blocking=True)
            print(f"[{self.name}] Generated new messages: {self.generated_new_messages}")
            self.state = "DICTATE"

        elif self.state == "DICTATE":
            self.send_message_to_agents(self.generated_new_messages)
            self.generated_new_messages = []
            self.most_recently_received_messages = []
            self.state = "SPECTATE"

        # print(f"[{self.name}] New State: {self.state}")

    def mutate_state(self, internal_state):
        internal_state.append({'mutated': True})
        print(f"[{self.name}] Internal state mutated")
        return internal_state

    def send_message_to_agents(self, messages):
        for agent in self.other_agents:
            agent.inbox.enqueue(messages)
            print(f"[{self.name}] Sent messages to {agent.name}: {[message['content'] for message in messages]}")

# Create two instances of the agent
agent_1 = Agent("Agent 1", get_prompt("manager"))
agent_2 = Agent("Agent 2", get_prompt("dev"))

# Add each agent to the other's list of agents
agent_1.add_other_agent(agent_2)
agent_2.add_other_agent(agent_1)

# Simulate receiving a message
agent_1.inbox.enqueue({"role": "user", 
                       "content": util_read_file("current_task.txt"),
                       "type": "message"})

# Run the state handler function to control the agents in a loop
for _ in range(12):
    agent_1.handle_state()
    agent_2.handle_state()


[Agent 1] Current State: SPECTATE
[Agent 1] Inbox Size: 1
[Agent 1] Received messages: [{'role': 'user', 'content': 'Current task:\nMake a new file called "file_{timestamp}.txt" where timestamp is unix timestamp.\nSave the file and create a new git commit on master branch with the changes.\nMake sure you are on master branch and commit to master branch\n', 'type': 'message'}]
[Agent 2] Current State: SPECTATE
[Agent 2] Inbox Size: 0
[Agent 1] Current State: MUTATE
[Agent 1] Inbox Size: 0
[Agent 1] Internal state after mutation: [{'role': 'user', 'content': 'Current task:\nMake a new file called "file_{timestamp}.txt" where timestamp is unix timestamp.\nSave the file and create a new git commit on master branch with the changes.\nMake sure you are on master branch and commit to master branch\n', 'type': 'message'}]


Output()

Output()

Output()

Output()

...

[Agent 2] Generated new messages: [{'role': 'assistant', 'type': 'message', 'content': 'It looks like we\'ve successfully completed the Git commit process for the new file. \n\n### Summary of Actions Completed:\n1. Added the file **"file_1721765804.txt"** to the staging area.\n2. Created a commit with the message: **"Add new file file_1721765804.txt"**.\n\nIf you have any further tasks or requests, please let me know!'}]
[Agent 1] Current State: SPECTATE
[Agent 1] Inbox Size: 0
[Agent 2] Current State: DICTATE
[Agent 2] Inbox Size: 0
[Agent 2] Sent messages to Agent 1: ['It looks like we\'ve successfully completed the Git commit process for the new file. \n\n### Summary of Actions Completed:\n1. Added the file **"file_1721765804.txt"** to the staging area.\n2. Created a commit with the message: **"Add new file file_1721765804.txt"**.\n\nIf you have any further tasks or requests, please let me know!']
[Agent 1] Current State: SPECTATE
[Agent 1] Inbox Size: 1
[Agent 1] Received messages:

Output()

Output()

Output()

[Agent 1] Generated new messages: [{'role': 'assistant', 'type': 'message', 'content': "Great job on completing the Git commit process for the new file!\n\n### Plan for Next Steps:\n1. Verify the status of the Git repository to ensure the commit was successful.\n2. If the status is as expected, consider pushing the changes to the remote repository.\n3. If applicable, pull the latest changes from the remote repository to ensure you have the most updated version.\n\nLet's start with the first step: verifying the status of the Git repository. I'll write a shell command to check the Git status. \n\nNow, I'll execute the command to check the status."}, {'role': 'assistant', 'type': 'code', 'format': 'shell', 'content': 'git status'}, {'role': 'computer', 'type': 'console', 'format': 'output', 'content': '\nOn branch master\nChanges not staged for commit:\n  (use "git add <file>..." to update what will be committed)\n  (use "git restore <file>..." to discard changes in working directory)\n\t

In [19]:
import openai

In [33]:
import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role":"system",
            "content": get_prompt("manager")
        },
        {
            "role": "user",
            "content": util_read_file("current_task.txt"),
        }
    ],
    model="gpt-4o-mini",
)

print(chat_completion.choices[0].message.content)

To complete the current task of creating a new file with the specified format and committing it to the master branch, please follow these step-by-step instructions:

### Instructions

1. **Verify Current Branch**: 
   First, ensure that you are on the master branch. Open your terminal or command line and run the following command:
   ```bash
   git branch
   ```
   - If you see `* master`, you are already on the master branch. If not, switch to the master branch using:
   ```bash
   git checkout master
   ```

2. **Create a Unix Timestamp**:
   In your terminal, you can generate a Unix timestamp using the following command:
   ```bash
   timestamp=$(date +%s)
   ```
   - This command will store the current Unix timestamp in the variable `timestamp`.

3. **Create the New File**:
   Next, create a new file with the name format `file_{timestamp}.txt`. Use the following command:
   ```bash
   touch file_${timestamp}.txt
   ```
   - Ensure that the filename appears correctly by running `ls`

In [27]:
chat_completion.choices[0].message.content

'This is a test. How can I assist you further?'